# Out-of-State-Contributions: National Analysis

How much out-of-state money have candidates nationally raised in the 2018 election cycle, in absolute and proportional terms, thus far and how does that compare with this point in the 2014 and 2010 cycles?

In [1]:
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import contributions data.

In [2]:
%%notify
contributions = pd.read_csv("data/contributions.csv")
contributions.info()

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,14,15,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362352 entries, 0 to 6362351
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      object
in_out_state              object
no_veto                   object
office                    object
latest_month              object
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: float64(1), int64(2), object(16)
memory usage: 922.3+ MB


<IPython.core.display.Javascript object>

Convert the contribution date and latest month columns to datetime data type.

In [3]:
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")
contributions["latest_month"] = pd.to_datetime(contributions["latest_month"], errors="coerce")
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362352 entries, 0 to 6362351
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 922.3+ MB


## Calculate out-of-state contributions by party and year

Group by year and party and in-vs.-out-of-state contribution status and sum contributions.

In [4]:
contributions_by_party = contributions.groupby(["year", "party", "in_out_state"])["amount"].sum().reset_index()
contributions_by_party.head()

,year,party,in_out_state,amount
0,2010,Democratic,in-state,"470,990,619.10"
1,2010,Democratic,out-of-state,"58,985,872.49"
2,2010,Democratic,unknown,"1,559,172.79"
3,2010,Nonpartisan,in-state,"941,726.00"
4,2010,Nonpartisan,out-of-state,"136,753.70"


Pivot dataframe to aggregate each party's data in a single row.

In [5]:
contributions_by_party = pd.pivot_table(contributions_by_party, index=["party"], columns=["year", "in_out_state"]).reset_index()
contributions_by_party

party         amount                             \
year                                2010                              
in_out_state                    in-state  out-of-state      unknown   
0              Democratic 470,990,619.10 58,985,872.49 1,559,172.79   
1             Nonpartisan     941,726.00    136,753.70   -19,688.72   
2              Republican 648,677,300.68 50,017,282.78   765,314.14   
3             Third-Party   6,187,102.85    653,764.02 1,536,886.63   
4                 Unknown            nan           nan          nan   

                                                                        \
year                   2014                                       2018   
in_out_state       in-state  out-of-state       unknown       in-state   
0            411,359,783.73 71,051,343.88  6,043,230.29 669,398,566.40   
1              1,617,096.45    156,743.05      4,068.00   1,376,337.33   
2            458,259,289.83 74,927,029.31 10,994,808.72 734,347,784.64   
3             12,241,033.92    618,352.81     37,163.76   3,816,576.12   
4                       nan           nan           nan      28,042.01   

                                          
year                                      
in_out_state  out-of-state       unknown  
0            96,373,778.02 10,174,647.15  
1               310,506.30     -3,156.23  
2            69,029,152.41  9,172,355.84  
3               399,793.60    562,243.33  
4                 7,944.09           nan

Some records have no contributions for certain categories. Let's set those values equal to zero to be sure any calculations we run on them are correct.

In [6]:
contributions_by_party.fillna(0, inplace=True)

Flatten the resulting dataframe's multi-index columns.

In [7]:
contributions_by_party.columns = ["party", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_party

,party,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,Democratic,"470,990,619.10","58,985,872.49","1,559,172.79","411,359,783.73","71,051,343.88","6,043,230.29","669,398,566.40","96,373,778.02","10,174,647.15"
1,Nonpartisan,"941,726.00","136,753.70","-19,688.72","1,617,096.45","156,743.05","4,068.00","1,376,337.33","310,506.30","-3,156.23"
2,Republican,"648,677,300.68","50,017,282.78","765,314.14","458,259,289.83","74,927,029.31","10,994,808.72","734,347,784.64","69,029,152.41","9,172,355.84"
3,Third-Party,"6,187,102.85","653,764.02","1,536,886.63","12,241,033.92","618,352.81","37,163.76","3,816,576.12","399,793.60","562,243.33"
4,Unknown,0.00,0.00,0.00,0.00,0.00,0.00,"28,042.01","7,944.09",0.00


Calculate the total contributions by cycle.

In [8]:
contributions_by_party["18_total"] = contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"]
contributions_by_party["14_total"] = contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"]
contributions_by_party["10_total"] = contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"]
contributions_by_party = contributions_by_party[["party", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [9]:
contributions_by_party["pct_18_in_state"] = contributions_by_party["18_in_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_out_of_state"] = contributions_by_party["18_out_of_state"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_18_unknown"] = contributions_by_party["18_unknown"] / (contributions_by_party["18_in_state"] + contributions_by_party["18_out_of_state"] + contributions_by_party["18_unknown"])
contributions_by_party["pct_14_in_state"] = contributions_by_party["14_in_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_out_of_state"] = contributions_by_party["14_out_of_state"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_14_unknown"] = contributions_by_party["14_unknown"] / (contributions_by_party["14_in_state"] + contributions_by_party["14_out_of_state"] + contributions_by_party["14_unknown"])
contributions_by_party["pct_10_in_state"] = contributions_by_party["10_in_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_out_of_state"] = contributions_by_party["10_out_of_state"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party["pct_10_unknown"] = contributions_by_party["10_unknown"] / (contributions_by_party["10_in_state"] + contributions_by_party["10_out_of_state"] + contributions_by_party["10_unknown"])
contributions_by_party

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,party,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,Democratic,"669,398,566.40","96,373,778.02","10,174,647.15","775,946,991.57","411,359,783.73","71,051,343.88","6,043,230.29","488,454,357.90","470,990,619.10","58,985,872.49","1,559,172.79","531,535,664.38",0.86,0.12,0.01,0.84,0.15,0.01,0.89,0.11,0.00
1,Nonpartisan,"1,376,337.33","310,506.30","-3,156.23","1,683,687.40","1,617,096.45","156,743.05","4,068.00","1,777,907.50","941,726.00","136,753.70","-19,688.72","1,058,790.98",0.82,0.18,-0.00,0.91,0.09,0.00,0.89,0.13,-0.02
2,Republican,"734,347,784.64","69,029,152.41","9,172,355.84","812,549,292.89","458,259,289.83","74,927,029.31","10,994,808.72","544,181,127.86","648,677,300.68","50,017,282.78","765,314.14","699,459,897.60",0.90,0.08,0.01,0.84,0.14,0.02,0.93,0.07,0.00
3,Third-Party,"3,816,576.12","399,793.60","562,243.33","4,778,613.05","12,241,033.92","618,352.81","37,163.76","12,896,550.49","6,187,102.85","653,764.02","1,536,886.63","8,377,753.50",0.80,0.08,0.12,0.95,0.05,0.00,0.74,0.08,0.18
4,Unknown,"28,042.01","7,944.09",0.00,"35,986.10",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.78,0.22,0.00,nan,nan,nan,nan,nan,nan


## Calculate out-of-state contributions by office and year

Group by year, standardized office and in-vs.-out-of-state contribution status and sum contributions.

In [10]:
contributions_by_office = contributions.groupby(["year", "standardized_office", "in_out_state"])["amount"].sum().reset_index()
contributions_by_office.head()

,year,standardized_office,in_out_state,amount
0,2010,GOVERNOR/LIEUTENANT GOVERNOR,in-state,"682,436,934.47"
1,2010,GOVERNOR/LIEUTENANT GOVERNOR,out-of-state,"66,096,466.69"
2,2010,GOVERNOR/LIEUTENANT GOVERNOR,unknown,"2,106,181.41"
3,2010,STATE HOUSE/ASSEMBLY,in-state,"275,793,268.81"
4,2010,STATE HOUSE/ASSEMBLY,out-of-state,"26,018,468.96"


Pivot the dataframe to aggregate each office's data in a single row.

In [11]:
contributions_by_office = pd.pivot_table(contributions_by_office, index=["standardized_office"], columns=["year", "in_out_state"]).reset_index()
contributions_by_office

standardized_office         amount                \
year                                                 2010                 
in_out_state                                     in-state  out-of-state   
0             GOVERNOR/LIEUTENANT GOVERNOR 682,436,934.47 66,096,466.69   
1                     STATE HOUSE/ASSEMBLY 275,793,268.81 26,018,468.96   
2                             STATE SENATE 168,566,545.35 17,678,737.34   

                                                                      \
year                                2014                               
in_out_state      unknown       in-state  out-of-state       unknown   
0            2,106,181.41 407,211,261.30 92,392,336.66 10,240,160.27   
1              997,719.04 299,530,302.74 34,655,932.13  4,656,823.01   
2              737,784.39 176,735,639.89 19,705,200.26  2,182,287.49   

                                                        
year                   2018                             
in_out_state       in-state  out-of-state      unknown  
0            893,307,028.24 98,147,131.47 7,690,866.40  
1            306,868,559.77 42,249,630.05 7,628,600.40  
2            208,791,718.49 25,724,412.90 4,586,623.29

Flatten the resulting dataframe's multi-index columns.

In [12]:
contributions_by_office.columns = ["standardized_office", "10_in_state", "10_out_of_state", "10_unknown",
                                  "14_in_state", "14_out_of_state", "14_unknown",
                                  "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_office

,standardized_office,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"682,436,934.47","66,096,466.69","2,106,181.41","407,211,261.30","92,392,336.66","10,240,160.27","893,307,028.24","98,147,131.47","7,690,866.40"
1,STATE HOUSE/ASSEMBLY,"275,793,268.81","26,018,468.96","997,719.04","299,530,302.74","34,655,932.13","4,656,823.01","306,868,559.77","42,249,630.05","7,628,600.40"
2,STATE SENATE,"168,566,545.35","17,678,737.34","737,784.39","176,735,639.89","19,705,200.26","2,182,287.49","208,791,718.49","25,724,412.90","4,586,623.29"


Calculate the total contributions by cycle.

In [13]:
contributions_by_office["18_total"] = contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"]
contributions_by_office["14_total"] = contributions_by_office["14_in_state"] + contributions_by_office["14_out_of_state"] + contributions_by_office["14_unknown"]
contributions_by_office["10_total"] = contributions_by_office["10_in_state"] + contributions_by_office["10_out_of_state"] + contributions_by_office["10_unknown"]
contributions_by_office = contributions_by_office[["standardized_office", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]
contributions_by_office

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total
0,GOVERNOR/LIEUTENANT GOVERNOR,"893,307,028.24","98,147,131.47","7,690,866.40","999,145,026.11","407,211,261.30","92,392,336.66","10,240,160.27","509,843,758.23","682,436,934.47","66,096,466.69","2,106,181.41","750,639,582.57"
1,STATE HOUSE/ASSEMBLY,"306,868,559.77","42,249,630.05","7,628,600.40","356,746,790.22","299,530,302.74","34,655,932.13","4,656,823.01","338,843,057.88","275,793,268.81","26,018,468.96","997,719.04","302,809,456.81"
2,STATE SENATE,"208,791,718.49","25,724,412.90","4,586,623.29","239,102,754.68","176,735,639.89","19,705,200.26","2,182,287.49","198,623,127.64","168,566,545.35","17,678,737.34","737,784.39","186,983,067.08"


Calculate the proportion of in-state, out-of-state and unknown contributions.

In [14]:
contributions_by_office["pct_18_in_state"] = contributions_by_office["18_in_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_out_of_state"] = contributions_by_office["18_out_of_state"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office["pct_18_unknown"] = contributions_by_office["18_unknown"] / (contributions_by_office["18_in_state"] + contributions_by_office["18_out_of_state"] + contributions_by_office["18_unknown"])
contributions_by_office

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,standardized_office,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown
0,GOVERNOR/LIEUTENANT GOVERNOR,"893,307,028.24","98,147,131.47","7,690,866.40","999,145,026.11","407,211,261.30","92,392,336.66","10,240,160.27","509,843,758.23","682,436,934.47","66,096,466.69","2,106,181.41","750,639,582.57",0.89,0.10,0.01
1,STATE HOUSE/ASSEMBLY,"306,868,559.77","42,249,630.05","7,628,600.40","356,746,790.22","299,530,302.74","34,655,932.13","4,656,823.01","338,843,057.88","275,793,268.81","26,018,468.96","997,719.04","302,809,456.81",0.86,0.12,0.02
2,STATE SENATE,"208,791,718.49","25,724,412.90","4,586,623.29","239,102,754.68","176,735,639.89","19,705,200.26","2,182,287.49","198,623,127.64","168,566,545.35","17,678,737.34","737,784.39","186,983,067.08",0.87,0.11,0.02


## Calculate 2018 out-of-state contributions by redistricting role

Filter the contributions data to the 2018 cycle.

In [15]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_18.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2441346 entries, 0 to 2441345
Data columns (total 19 columns):
candidate                 object
candidate_id              int64
year                      int64
state                     object
party                     object
election_status           object
contributor               object
amount                    float64
date                      datetime64[ns]
in_out_state              object
no_veto                   object
office                    object
latest_month              datetime64[ns]
redistricting_role        object
independent_commission    object
single_house_district     object
standardized_office       object
standardized_status       object
two_year_term             object
dtypes: datetime64[ns](2), float64(1), int64(2), object(14)
memory usage: 372.5+ MB


Group by redistricting role and in-vs.-out-of-state contribution status and sum contributions.

In [16]:
contributions_by_redistricting = contributions.groupby(["redistricting_role", "in_out_state"])["amount"].sum().reset_index()
contributions_by_redistricting

,redistricting_role,in_out_state,amount
0,N,in-state,"236,701,449.38"
1,N,out-of-state,"28,458,259.52"
2,N,unknown,"4,255,776.97"
3,Y,in-state,"1,172,265,857.12"
4,Y,out-of-state,"137,662,914.90"
5,Y,unknown,"15,650,313.12"


Pivot dataframe to aggregate each role's data in a single row.

In [17]:
contributions_by_redistricting = pd.pivot_table(contributions_by_redistricting, index=["redistricting_role"], columns=["in_out_state"]).reset_index()
contributions_by_redistricting

redistricting_role           amount                             
in_out_state                            in-state   out-of-state       unknown
0                             N   236,701,449.38  28,458,259.52  4,255,776.97
1                             Y 1,172,265,857.12 137,662,914.90 15,650,313.12

Flatten the resulting dataframe's multi-index columns.

In [18]:
contributions_by_redistricting.columns = ["redistricting_role", "in_state", "out_of_state", "unknown"]
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown
0,N,"236,701,449.38","28,458,259.52","4,255,776.97"
1,Y,"1,172,265,857.12","137,662,914.90","15,650,313.12"


Calculate the total contributions by redistricting role.

In [19]:
contributions_by_redistricting["total"] = contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"]

Calculate the proportion of in-state, out-of-state and unknown contributions by redistricting role.

In [20]:
contributions_by_redistricting["pct_in_state"] = contributions_by_redistricting["in_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_out_of_state"] = contributions_by_redistricting["out_of_state"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting["pct_unknown"] = contributions_by_redistricting["unknown"] / (contributions_by_redistricting["in_state"] + contributions_by_redistricting["out_of_state"] + contributions_by_redistricting["unknown"])
contributions_by_redistricting

,redistricting_role,in_state,out_of_state,unknown,total,pct_in_state,pct_out_of_state,pct_unknown
0,N,"236,701,449.38","28,458,259.52","4,255,776.97","269,415,485.87",0.88,0.11,0.02
1,Y,"1,172,265,857.12","137,662,914.90","15,650,313.12","1,325,579,085.14",0.88,0.10,0.01


## Calculate contributions by candidate status and year

Group by year and candidate status and in-vs.-out-of-state contribution status and sum contributions.

In [21]:
contributions_by_status = contributions.groupby(["year", "standardized_status", "in_out_state"])["amount"].sum().reset_index()
contributions_by_status

,year,standardized_status,in_out_state,amount
0,2010,ADVANCED TO GENERAL,in-state,"884,558,754.08"
1,2010,ADVANCED TO GENERAL,out-of-state,"92,744,539.62"
2,2010,ADVANCED TO GENERAL,unknown,"3,123,673.10"
3,2010,DID NOT ADVANCE,in-state,"242,237,994.55"
4,2010,DID NOT ADVANCE,out-of-state,"17,049,133.37"
5,2010,DID NOT ADVANCE,unknown,"718,011.74"
6,2014,ADVANCED TO GENERAL,in-state,"753,996,250.29"
7,2014,ADVANCED TO GENERAL,out-of-state,"127,702,719.04"
8,2014,ADVANCED TO GENERAL,unknown,"11,805,094.42"
9,2014,DID NOT ADVANCE,in-state,"129,480,953.64"


Pivot dataframe to aggregate each candidate status' data in a single row.

In [22]:
contributions_by_status = pd.pivot_table(contributions_by_status, index=["standardized_status"], columns=["year", "in_out_state"]).reset_index()
contributions_by_status

standardized_status         amount                             \
year                                        2010                              
in_out_state                            in-state  out-of-state      unknown   
0             ADVANCED TO GENERAL 884,558,754.08 92,744,539.62 3,123,673.10   
1                 DID NOT ADVANCE 242,237,994.55 17,049,133.37   718,011.74   

                                                                           \
year                   2014                                          2018   
in_out_state       in-state   out-of-state       unknown         in-state   
0            753,996,250.29 127,702,719.04 11,805,094.42 1,012,608,092.73   
1            129,480,953.64  19,050,750.01  5,274,176.35   396,359,213.77   

                                           
year                                       
in_out_state   out-of-state       unknown  
0            129,061,194.54 15,160,609.36  
1             37,059,979.88  4,745,480.73

Flatten the resulting dataframe's multi-index columns.

In [23]:
contributions_by_status.columns = ["standardized_status", "10_in_state", "10_out_of_state", "10_unknown",
                                   "14_in_state", "14_out_of_state", "14_unknown",
                                   "18_in_state", "18_out_of_state", "18_unknown"
                                  ]
contributions_by_status

,standardized_status,10_in_state,10_out_of_state,10_unknown,14_in_state,14_out_of_state,14_unknown,18_in_state,18_out_of_state,18_unknown
0,ADVANCED TO GENERAL,"884,558,754.08","92,744,539.62","3,123,673.10","753,996,250.29","127,702,719.04","11,805,094.42","1,012,608,092.73","129,061,194.54","15,160,609.36"
1,DID NOT ADVANCE,"242,237,994.55","17,049,133.37","718,011.74","129,480,953.64","19,050,750.01","5,274,176.35","396,359,213.77","37,059,979.88","4,745,480.73"


Calculate the total contributions by cycle.

In [24]:
contributions_by_status["18_total"] = contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"]
contributions_by_status["14_total"] = contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"]
contributions_by_status["10_total"] = contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"]
contributions_by_status = contributions_by_status[["standardized_status", "18_in_state", "18_out_of_state", "18_unknown", "18_total", "14_in_state", "14_out_of_state", "14_unknown", "14_total", "10_in_state", "10_out_of_state", "10_unknown", "10_total"]]

Calculate the proportion of in-state, out-of-state and unknown contributions by cycle.

In [25]:
contributions_by_status["pct_18_in_state"] = contributions_by_status["18_in_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_out_of_state"] = contributions_by_status["18_out_of_state"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_18_unknown"] = contributions_by_status["18_unknown"] / (contributions_by_status["18_in_state"] + contributions_by_status["18_out_of_state"] + contributions_by_status["18_unknown"])
contributions_by_status["pct_14_in_state"] = contributions_by_status["14_in_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_out_of_state"] = contributions_by_status["14_out_of_state"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_14_unknown"] = contributions_by_status["14_unknown"] / (contributions_by_status["14_in_state"] + contributions_by_status["14_out_of_state"] + contributions_by_status["14_unknown"])
contributions_by_status["pct_10_in_state"] = contributions_by_status["10_in_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_out_of_state"] = contributions_by_status["10_out_of_state"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status["pct_10_unknown"] = contributions_by_status["10_unknown"] / (contributions_by_status["10_in_state"] + contributions_by_status["10_out_of_state"] + contributions_by_status["10_unknown"])
contributions_by_status

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

,standardized_status,18_in_state,18_out_of_state,18_unknown,18_total,14_in_state,14_out_of_state,14_unknown,14_total,10_in_state,10_out_of_state,10_unknown,10_total,pct_18_in_state,pct_18_out_of_state,pct_18_unknown,pct_14_in_state,pct_14_out_of_state,pct_14_unknown,pct_10_in_state,pct_10_out_of_state,pct_10_unknown
0,ADVANCED TO GENERAL,"1,012,608,092.73","129,061,194.54","15,160,609.36","1,156,829,896.63","753,996,250.29","127,702,719.04","11,805,094.42","893,504,063.75","884,558,754.08","92,744,539.62","3,123,673.10","980,426,966.80",0.88,0.11,0.01,0.84,0.14,0.01,0.90,0.09,0.00
1,DID NOT ADVANCE,"396,359,213.77","37,059,979.88","4,745,480.73","438,164,674.38","129,480,953.64","19,050,750.01","5,274,176.35","153,805,880.00","242,237,994.55","17,049,133.37","718,011.74","260,005,139.66",0.90,0.08,0.01,0.84,0.12,0.03,0.93,0.07,0.00


## Export the data

In [26]:
%%notify
writer = pd.ExcelWriter("data/national_analysis.xlsx")
contributions_by_party.to_excel(writer, "contributions_by_party", index=False)
contributions_by_office.to_excel(writer, "contributions_by_office", index=False)
contributions_by_redistricting.to_excel(writer, "contributions_by_redistricting", index=False)
contributions_by_status.to_excel(writer, "contributions_by_status", index=False)
writer.save()